# BERT를 이용한 네이버 영화 리뷰 분석
- BERT는 Transformer의 encoder를 사용합니다.
- Transformer 구현에 대해 이미 알아보았으니, BERT를 직접 구현하지 않고 huggingface 라이브러리를 통해 간단하게 구현하는 방법에 대해 배우도록 하겠습니다.
- BERT를 이용해서는 naver 영화리뷰 데이터의 점수를 분류하는 task를 진행해보겠습니다.
- https://huggingface.co/

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
import time
import torch
import torch.nn as nn
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 53.1 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 3.3 MB 13.1 MB/s 
     |████████████████████████████████| 596 kB 52.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


- BERT tokenizer와 model을 이용해 네이버 영화리뷰 데이터에 fine-tuning을 진행해봅시다.

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")
naver_df = pd.read_table('ratings.txt')
naver_df = naver_df.dropna(how='any')
with open('naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))

In [ ]:
naver_df.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [ ]:
naver_df.tail()

,id,document,label
199995,8963373,포켓 몬스터 짜가 ㅡㅡ;;,0
199996,3302770,쓰.레.기,0
199997,5458175,완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.,0
199998,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0
199999,8548411,포풍저그가나가신다영차영차영차,0


In [ ]:
naver_df['label'].unique()

array([1, 0])

Pretain된 huggingface 라이브러리를 불러와 사용하였습니다.

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased') # multilingual-BERT를 사용해보겠습니다.
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')

inputs = tokenizer.encode("Hello, my dog is cute", return_tensors="pt")
outputs = model(inputs)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [ ]:
outputs

SequenceClassifierOutput([('logits',
                           tensor([[-0.2494,  0.3147]], grad_fn=<AddmmBackward0>))])

In [ ]:
inputs.shape

torch.Size([1, 9])

In [ ]:
# label을 입력해주면 classification에 대한 loss도 자동으로 계산할 수 있습니다.
model(inputs, labels=torch.tensor([1]))

SequenceClassifierOutput([('loss', tensor(0.4504, grad_fn=<NllLossBackward0>)),
                          ('logits',
                           tensor([[-0.2494,  0.3147]], grad_fn=<AddmmBackward0>))])

In [ ]:
train_data_idx = np.random.choice(range(len(naver_df)), size=len(naver_df)//5*4, replace=False)

In [ ]:
train_data_idx

array([ 83185, 130335,  27641, ...,  55761,  98927,  70444])

In [ ]:
train_data = naver_df.iloc[train_data_idx][['document','label']].values
test_data = naver_df.iloc[~naver_df.index.isin(train_data_idx)][['document','label']].values

In [ ]:
from torch.utils.data import Dataset
class ReviewDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x, y = self.data[idx]
        # BERT tokenizer는 batch 단위로 한번에 token화 할 수 있습니다.
        # dataloder에서 얻어온 후 한번에 tokenize 해보도록 하겠습니다.
        # text는 list로, label은 long tensor로 리턴해줍니다.
        return x, torch.tensor(y).long()

In [ ]:
train_dataset = ReviewDataset(train_data)
test_dataset = ReviewDataset(test_data)

In [ ]:
from torch.utils.data import DataLoader
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                         num_workers=2, collate_fn=None,
                          pin_memory=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, 
                         num_workers=2, collate_fn=None,
                          pin_memory=True, drop_last=False)

In [ ]:
x, y = next(iter(train_loader))

In [ ]:
x

['흑인들의 삶과 비애, 그리고 그들의 영혼이 담긴 랩..',
 '또 보고싶다.. 어디서 상영 안하나',
 '기분좋게 볼 수 있는 영화~영화란 이렇게!',
 '이건 욕이 안나올수가 없다...태어나서 처음으로 로빈윌리엄스의 모습을 스크린에서 뵙고 싶었는데 몇.몇 영화들의 상영관독점으로..',
 '인디아나존스의 냄새만 풍기는 홍콩판 보물찻기',
 '대안없는 비판.. 까는 건 쉽지..',
 '처음 볼 때만 재밌는 클리셰적인 킬링타임용 영화.',
 '말이 필요없죠ㅠㅠ',
 '스파이 정말 재미있게 보고잇 있습니다~',
 '근데 남자성형도 많다는거..결혼으로보지않고. 성격 외향 다보고.. 더더군다나.. 성격 거지같은 것들끼리 만났으면.좋겠어요.. 그래서 바로깨우치고 파탄하겠죠..ㅎㅎ어린아이들과 노약자에게 행을 베풀고 양버할줄아는 그런 청년 성년 노년 이 되요 ㅎㅎ',
 '아아 또보고 싶다....ㅠㅠ',
 '베오울프 신화의 재해석! 끈질긴 연구와 열정, 추적으로 괴물들을 척결한다!!',
 '완전 뽀르노넹.. 절대 이해 안 된다.',
 '이 행복감',
 '너무 잔인하고 스토리 구성도 빈약하다 15세가 아니라 19세로 해야 할 불필요하게 잔인한 영화다 이런 수준으로밖에 만들지 못하는 것은 감독의 능력부족으로보인다',
 '마음이 너무 아픈 영화였습니다. 왜 세상은 바뀌지 않는 것인가요.',
 '따뜻한 영화! 봐도 봐도 재밌어요',
 '약 2시간동안 내가 뭘 본건지... 고은누나 얼굴만 본듯',
 '재밌다 몰입도 굿',
 '나이스',
 '자알보았습니다아..',
 '보고 또 보고.. 리메이크작이냐..-0-???',
 '극장에서봤지만 내용이 기억이 안남. 재미없게 봤을것이 분명.',
 '그렇게 소녀는 커간다..',
 '기자평론가들은 영화를 보고 남긴거 맞나? ㅋㅋㅋㅋ',
 '투자대비 효율 제로에 가까운 가성비 최악의 영화. 화려한 캐스팅과 드라마의 인지도에도 불구하고 반에 반에 반타작도 못한 망작영화.',
 '오ㅓㅏ',
 '다시 공평하게 시작하시죠',
 '심리건 뭐건 필요없다.. 스

In [ ]:
y

tensor([1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1,
        1, 0, 1, 0, 0, 1, 0, 1])

In [ ]:
encoded_x = tokenizer.batch_encode_plus(x, padding=True, return_tensors='pt')

In [ ]:
encoded_x.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
encoded_x['input_ids']

tensor([[  101, 10016, 12030,  ...,     0,     0,     0],
        [  101,  9144, 98199,  ...,     0,     0,     0],
        [  101,  8932, 37712,  ...,     0,     0,     0],
        ...,
        [  101, 23545, 33323,  ...,     0,     0,     0],
        [  101,  9519, 10739,  ...,     0,     0,     0],
        [  101,  9786, 96720,  ...,     0,     0,     0]])

In [ ]:
encoded_x['token_type_ids']

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [ ]:
encoded_x['attention_mask']

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

In [ ]:
def train_epoch(model, dataloader, tokenizer, optimizer):
    model.train()
    train_loss = 0
    for i, (x,y) in enumerate(dataloader):
        x = tokenizer.batch_encode_plus(x, padding=True, return_tensors='pt')['input_ids'].to(DEVICE)
        y = y.to(DEVICE)
        loss = model(x, labels=y)['loss']
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        if i % 50 == 0:
            print('Iter [{}/{}] Loss {:.6f}'.format(i+1, len(dataloader), train_loss / (i+1)))
    
    return train_loss / len(dataloader)

def test_epoch(model, dataloader, tokenizer):
    model.eval()
    preds = []
    labels = []
    with torch.no_grad():
      for x,y in dataloader:
          x = tokenizer.batch_encode_plus(x, padding=True, return_tensors='pt')['input_ids'].to(DEVICE)
          out = model(x)['logits']
          pred = out.argmax(-1)
          preds.append(pred.cpu())
          labels.append(y)
    preds = torch.cat(preds)
    labels = torch.cat(labels)
    acc = (preds == labels).float().mean()
    print('ACC : {:.3f}'.format(acc))
    return preds, labels

def predict(model, tokenizer, sentence):
    model.eval()
    x = tokenizer.encode(sentence, return_tensors='pt').to(DEVICE)
    out = model(x)['logits']
    pred = out.argmax(-1)
    return pred.cpu()

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [ ]:
EPOCHS=1

for i in range(EPOCHS):
    train_epoch(model, train_loader, tokenizer, optimizer)
    test_epoch(model, test_loader, tokenizer)

Iter [1/4999] Loss 0.693272
Iter [51/4999] Loss 0.695850
Iter [101/4999] Loss 0.683021
Iter [151/4999] Loss 0.666095
Iter [201/4999] Loss 0.638034


In [ ]:
predict(model, tokenizer, '이 영화는 최고야')

tensor([1])

In [ ]:
predict(model, tokenizer, '이 영화는 별로야')

tensor([0])

In [ ]:
predict(model, tokenizer, '이 영화는 감동적이야')

tensor([1])

In [ ]:
predict(model, tokenizer, '영화 정말 잘 만들었어!')

tensor([1])

In [ ]:
predict(model, tokenizer, '출연진 때문에 안봐')

tensor([0])